# CIS545: Final Project - CORD-19 Dataset
## Justin Choi
## TA: Hoyt Gong

Hi there! For this project, I opted to use the COVID-19 research challenge dataset, which they named "CORD-19" (I promise, the title wasn't a typo haha), and below you'll see the respective EDA and modeling I've done for the dataset! Hope you enjoy ~


In [1]:
import pandas as pd 
import numpy as np 
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import glob, os
import json
import nltk


In [2]:
metadata_df = pd.read_csv('./CORD-19-research-challenge/metadata.csv')

# Import all the json files
cord_19_folder = './CORD-19-research-challenge/'
list_of_files = []; # only going to take those from pdf_json! not pmc_json
for root, dirs, files in os.walk(cord_19_folder):
    for name in files:
        if (name.endswith('.json')):
            full_path = os.path.join(root, name)
            list_of_files.append(full_path)
sorted(list_of_files)
print('done')

# ALTERNATE

# all_json = glob.glob(f'{cord_19_folder}/**/*.json', recursive=True)
# len(all_json)


done


In [5]:
class JsonReader:
    def __init__(self, file_path):
        with open(file_path) as file: 
            content = json.load(file)
            
            # start to insert the abstract and body text 
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = [] 
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.body_text = '\n'.join(self.body_text)
    def __repr__(self):
        return f'{self.paper_id}: {self.body_text[:500]}...'

random_json = list_of_files[47404]
sample_article = JsonReader(random_json)
print(sample_article)

PMC1403753: Migration, transmigration, [1] return migration, and remigration constitute defining elements of the current and future world order. More than 700 million people (including visitors on business or personal/family trips) traverse nation-state borders annually [2,3] and one million per week move between the global South and the global North [4]. The enormity of contemporary transnational mobility is illustrated by the case of Australia. In the past half century, Australia's "resident population ha...


In [6]:
input = {'paper_id': [], 'doi':[], 'abstract': [], 'body_text': [], 'authors': [], 'title': [], 'journal': []}

for index, entry in enumerate(list_of_files):
    if index % (len(list_of_files) // 10) ==    0:
        print(f'Processing {index} of {len(list_of_files)}')
    try: 
        article = JsonReader(entry)
    except Exception as e: 
        continue #means that we don't have a valid file format  
    
    metadata = metadata_df.loc[metadata_df['sha'] == article.paper_id]
    if len(metadata) == 0:
        continue # no such metadata for paper in our csv, skip

    input['body_text'].append(article.body_text)
    input['paper_id'].append(article.paper_id)

    # add in metadata 
    title = metadata_df['title'].values[0] 
    doi = metadata_df['doi'].values[0] 
    abstract = metadata_df['abstract'].values[0] 
    authors = metadata_df['authors'].values[0] 
    journal = metadata_df['journal'].values[0] 

    input['title'].append(title)
    input['doi'].append(doi)
    input['abstract'].append(abstract)
    input['authors'].append(authors)
    input['journal'].append(journal)



Processing 0 of 59311
Processing 5931 of 59311
Processing 11862 of 59311
Processing 17793 of 59311
Processing 23724 of 59311
Processing 29655 of 59311
Processing 35586 of 59311
Processing 41517 of 59311
Processing 47448 of 59311
Processing 53379 of 59311
Processing 59310 of 59311


In [7]:
covid_df = pd.DataFrame(input, columns=['paper_id', 'doi', 'abstract', 'body_text', 'authors', 'title', 'journal'])
covid_df.head()

,paper_id,doi,abstract,body_text,authors,title,journal
0,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...",The Middle East respiratory syndrome coronavir...,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...","The 2019 novel coronavirus (SARS-CoV-2), ident...","Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health
2,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...",The evolutionary history of humans is characte...,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health
3,6599ebbef3d868afac9daa4f80fa075675cf03bc,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...","Sixty years ago, civil aviation was an infant ...","Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health
4,eb5c7f3ff921ad6469b79cc8a3c122648204ece4,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...",Acute infections of the gastrointestinal tract...,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health


In [8]:
covid_df['abstract_word_count'] = covid_df['abstract'].apply(lambda x : len(x.strip().split()))
covid_df['body_word_count'] = covid_df['body_text'].apply(lambda x : len(x.strip().split()))
covid_df['body_unique_count'] = covid_df['body_text'].apply(lambda x : len(set(x.strip().split())))
covid_df.head()

,paper_id,doi,abstract,body_text,authors,title,journal,abstract_word_count,body_word_count,body_unique_count
0,4ed70c27f14b7f9e6219fe605eae2b21a229f23c,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...",The Middle East respiratory syndrome coronavir...,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health,240,2748,996
1,306ef95a3a91e13a93bcc37fb2c509b67c0b5640,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...","The 2019 novel coronavirus (SARS-CoV-2), ident...","Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health,240,944,486
2,ab680d5dbc4f51252da3473109a7885dd6b5eb6f,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...",The evolutionary history of humans is characte...,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health,240,2884,1091
3,6599ebbef3d868afac9daa4f80fa075675cf03bc,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...","Sixty years ago, civil aviation was an infant ...","Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health,240,5838,1587
4,eb5c7f3ff921ad6469b79cc8a3c122648204ece4,10.1186/1471-2458-3-5,"BACKGROUND: Rhinovirus, the most common cause ...",Acute infections of the gastrointestinal tract...,"Myatt, Theodore A; Johnston, Sebastian L; Rudn...",Airborne rhinovirus detection and effect of ul...,BMC Public Health,240,6972,2249
